In [6]:
import csv
from pyproj import Proj, transform
import pandas as pd

# Definir los sistemas de referencia de entrada y salida
crs_4326 = Proj(init='epsg:4326')  # WGS84
crs_32620 = Proj(init='epsg:32620')  # WGS 84 / UTM zone 20S

# Archivos de entrada y salida
input_file = 'data/posiciones_ovitrampas2.csv'
output_file = 'data/datos_transformados.csv'

with open(input_file, 'r', newline='', encoding='utf-8') as csvfile, \
     open(output_file, 'w', newline='', encoding='utf-8') as outfile:

    reader = csv.reader(csvfile)
    writer = csv.writer(outfile)

    # Escribir la cabecera en el archivo de salida
    header = next(reader)
    writer.writerow(header)

    for row in reader:
        # Extraer datos del CSV
        id = row[0]
        lat = float(row[1])
        lon = float(row[2])
        fecha = row[3]

        # Transformar coordenadas de EPSG:4326 a EPSG:32620
        x, y = transform(crs_4326, crs_32620, lon, lat)

        # Escribir la fila transformada en el archivo de salida
        transformed_row = [id, y, x, fecha]  # Cambiar orden de latitud y longitud
        writer.writerow(transformed_row)

print(f'Se han transformado las coordenadas y guardado en {output_file}')


# Load the data from the CSV file
data = pd.read_csv(output_file)

# Calcular las coordenadas mínimas y máximas de latitud y longitud
min_lat = data['lat'].min()
max_lat = data['lat'].max()
min_lon = data['long'].min()
max_lon = data['long'].max()

# Calcular el centro del ROI
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

# Calcular la diferencia entre las coordenadas máximas y mínimas
diff_lat = max_lat - min_lat
diff_lon = max_lon - min_lon

# Agregar un margen de 2 km a la diferencia
margin = 2000  # 2 km en grados
diff_lat += margin
diff_lon += margin

# Calcular la mitad de la diferencia ajustada
half_diff_lat = diff_lat / 2
half_diff_lon = diff_lon / 2

if half_diff_lat > half_diff_lon:
	half_diff_lon = half_diff_lat
else:
	half_diff_lat = half_diff_lon

print(half_diff_lat == half_diff_lon)

displacement = half_diff_lat

# Calcular las coordenadas del ROI
roi_lat = [center_lat - half_diff_lat, center_lat + half_diff_lat]
roi_lon = [center_lon - half_diff_lon, center_lon + half_diff_lon]



# print(displacement)

# Definir las coordenadas de los vértices del polígono
polygon_points = [
    (center_lon - displacement, center_lat - displacement),
    (center_lon + displacement, center_lat - displacement),
    (center_lon + displacement, center_lat + displacement),
    (center_lon - displacement, center_lat + displacement),
    (center_lon - displacement, center_lat - displacement)  # Cerrar el polígono
]

# Crear el objeto de polígono usando shapely
polygon = Polygon(polygon_points)

# Crear un GeoDataFrame con el polígono
gdf = gpd.GeoDataFrame({'geometry': [polygon]})

gdf.to_csv("data/gdf.csv")

# Guardar el GeoDataFrame como un archivo Shapefile
output_shapefile = 'data/ROI/poligono.shp'
gdf.to_file(output_shapefile)


/usr/lib/python3/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/usr/lib/python3/dist-packages/pyproj/crs/crs.py:131: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/tmp/ipykernel_15710/123638438.py:31: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x, y = transform(crs_4326, crs_32620, lon, lat)


Se han transformado las coordenadas y guardado en datos_transformados.csv
True
